In [1]:
import os
import pyciemss
import torch
import pandas as pd
import numpy as np
from typing import Dict, List, Callable

import pyciemss.visuals.plots as plots
import pyciemss.visuals.vega as vega
import pyciemss.visuals.trajectories as trajectories

from pyciemss.integration_utils.intervention_builder import (
    param_value_objective,
    start_time_objective,
)

smoke_test = ('CI' in os.environ)

In [ ]:
Question 7 TODO: missing model

Setup for Scenario 1 question 9

In [5]:
MODEL_PATH = "https://raw.githubusercontent.com/gyorilab/mira/main/notebooks/evaluation_2024.03/epi_scenario1/sir_3strain_petrinet.json"

LOCAL_MODEL_PATH = "sir_3strain_petrinet.json"

start_time = 0.0
end_time = 62.0
logging_step_size = 1.0
num_samples =  10

In [18]:
import json
from mira.metamodel import *
from mira.modeling.amr.petrinet import template_model_to_petrinet_json
from mira.sources.amr.petrinet import template_model_from_amr_json
import sympy
with open(LOCAL_MODEL_PATH, 'r') as fh:
    tm = template_model_from_amr_json(json.load(fh))

c = lambda x: Concept(name=x, units=Unit(expression=s('person')))
s = lambda x: sympy.Symbol(x)

#dir(tm.parameters)

#print(tm.initials)

print(tm.parameters)

# names = ['infected_population_alpha_delta',
#          'immune_population_alpha_delta',
#           'infected_population_omicron_delta',
#           'infected_population_alpha_omicron_delta',
#           'immune_population_delta_omicron',
#           "immune_population_alpha_omicron",
#           "immune_population_alpha_delta_omicron",
#           "infected_population_delta_alpha",

#           ]

#for name in names:
#    tm.initials[name] = Initial(concept=c(name), expression = 0)

#print(tm.initials['infected_population_alpha_delta'])


{'beta_0': Parameter(name='beta_0', display_name=None, description=None, identifiers={}, context={}, units=Unit(expression=1/(day*person)), value=0.1, distribution=None), 'beta_1': Parameter(name='beta_1', display_name=None, description=None, identifiers={}, context={}, units=Unit(expression=1/(day*person)), value=0.1, distribution=None), 'beta_2': Parameter(name='beta_2', display_name=None, description=None, identifiers={}, context={}, units=Unit(expression=1/(day*person)), value=0.1, distribution=None), 'gamma_0': Parameter(name='gamma_0', display_name=None, description=None, identifiers={}, context={}, units=None, value=0.2, distribution=None), 'gamma_1': Parameter(name='gamma_1', display_name=None, description=None, identifiers={}, context={}, units=None, value=0.2, distribution=None), 'gamma_2': Parameter(name='gamma_2', display_name=None, description=None, identifiers={}, context={}, units=None, value=0.2, distribution=None), 'beta_alpha_delta': Parameter(name='beta_alpha_delta',

In [7]:


result = pyciemss.sample(LOCAL_MODEL_PATH, end_time, logging_step_size, num_samples, start_time=start_time)
display(result['data'].head())

ERROR:root:
                ###############################

                There was an exception in pyciemss

                Error occured in function: sample

                Function docs : 
    Load a model from a file, compile it into a probabilistic program, and sample from it.

    Args:
        model_path_or_json: Union[str, Dict]
            - A path to a AMR model file or JSON containing a model in AMR form.
        end_time: float
            - The end time of the sampled simulation.
        logging_step_size: float
            - The step size to use for logging the trajectory.
        num_samples: int
            - The number of samples to draw from the model.
        solver_method: str
            - The method to use for solving the ODE. See torchdiffeq's `odeint` method for more details.
            - If performance is incredibly slow, we suggest using `euler` to debug.
              If using `euler` results in faster simulation, the issue is likely that the model is s

ValueError: The model initial state could not be compiled. Please check the model definition.